In [43]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder

!pip install mlxtend==0.23.1

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**Joyceline Andriany Suryanto - 0706022210025**

# Association Rule for Store Dataset

In this case study, we will explore how association rule can be used to analyze the items that are usualy purcased together.

you can refer to this article to find out about apriori and association rule:
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

## Load Data

We will use the dataset of the transaction in a certain store. You can get the dataset here:
https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv

In [44]:
# load the data set ans show the first five transaction
df = pd.read_csv('https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv')
df.head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


# Get the set of product that has been purchased


In [45]:
list = df.unstack().unique()
list

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['Bread', 'Cheese', 'Meat', 'Eggs', 'Wine', 'Bagel', 'Pencil',
       'Diaper', 'Milk', nan], dtype=object)

## Preprocess Data

In this step, we will transform our dataset so that we will have a one hot encoding based on the purchased products.

In [46]:
#create an itemset based on the products

items = {item: 0 for item in list}

# encoding the feature
for item in df.iloc[0]:
  if item in items:
    items[item]=1

items


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'Bread': 1,
 'Cheese': 1,
 'Meat': 1,
 'Eggs': 1,
 'Wine': 1,
 'Bagel': 0,
 'Pencil': 1,
 'Diaper': 1,
 'Milk': 0,
 nan: 0}

In [47]:
# Replace NaN with a specific value, e.g., 'No Product'
df_with_no_product = df.fillna('No Product')

# Flatten the data again, but keep 'No Product' as a category
flattened_data_no_product = df_with_no_product.values.flatten()

# Step 1: Use OneHotEncoder, including 'No Product' (which represents missing values)
# Replace 'sparse' with 'sparse_output'
encoder_no_product = OneHotEncoder(sparse_output=False, handle_unknown='ignore')  # Make sure unknown values are ignored
encoded_no_product_data = encoder_no_product.fit_transform(flattened_data_no_product.reshape(-1, 1))

# Step 2: Create the one-hot encoded DataFrame
encoded_df_no_product = pd.DataFrame(encoded_no_product_data, columns=encoder_no_product.categories_[0])

# Step 3: Create a DataFrame with 0s and 1s, representing whether the item is bought
product_data_final = pd.DataFrame(0, index=df.index, columns=encoder_no_product.categories_[0])

# Loop through each row and mark presence of products, including 'No Product'
for i, row in df_with_no_product.iterrows():
    for product in row:
        product_data_final.loc[i, product] = 1

# Display the final DataFrame
print("Final one-hot encoded dataset with 'No Product':")
product_data_final.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Final one-hot encoded dataset with 'No Product':


,Bagel,Bread,Cheese,Diaper,Eggs,Meat,Milk,No Product,Pencil,Wine
0,0,1,1,1,1,1,0,0,1,1
1,0,1,1,1,0,1,1,0,1,1
2,0,0,1,0,1,1,1,1,0,1
3,0,0,1,0,1,1,1,1,0,1
4,0,0,0,0,0,1,0,1,1,1


In [48]:
# Since, the encoded dataframe consist of the empty column. We will drop the NaN column or u can use the index.
if 'No Product' in product_data_final.columns:
    product_data_final.drop(columns=['No Product'], inplace=True)

# Display the final DataFrame after dropping the 'No Product' column
product_data_final.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Bagel,Bread,Cheese,Diaper,Eggs,Meat,Milk,Pencil,Wine
0,0,1,1,1,1,1,0,1,1
1,0,1,1,1,0,1,1,1,1
2,0,0,1,0,1,1,1,0,1
3,0,0,1,0,1,1,1,0,1
4,0,0,0,0,0,1,0,1,1


Since, the encoded dataframe consist of the empty column. We will drop the NaN column or select all columns other than the first column.

## Apriori Algorithm

We will use appriori algorithm to determine the frequently purchased products.
For this case study, we will min_support=0.2

In [49]:
from mlxtend.frequent_patterns import apriori, association_rules

# Set a threshold value for support
min_support_value = 0.2  # You can adjust this value as needed

# Use apriori algorithm to get frequent itemsets
frequent_itemsets_result = apriori(product_data_final, min_support=min_support_value, use_colnames=True)

# Display the frequent itemsets
frequent_itemsets_result


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.425397,(Bagel)
1,0.504762,(Bread)
2,0.501587,(Cheese)
3,0.406349,(Diaper)
4,0.438095,(Eggs)
5,0.476190,(Meat)
6,0.501587,(Milk)
7,0.361905,(Pencil)
8,0.438095,(Wine)
9,0.279365,"(Bread, Bagel)"


The we will generate association rule of the frequent itemset based on confidence level with the threshold=0.6

In [51]:
confidence_threshold = 0.6
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=confidence_threshold)
rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Bagel),(Bread),0.425397,0.504762,0.279365,0.656716,1.301042,0.064641,1.442650,0.402687
1,(Bagel),(NaN),0.425397,0.869841,0.336508,0.791045,0.909413,-0.033520,0.622902,-0.147743
2,(Bread),(NaN),0.504762,0.869841,0.396825,0.786164,0.903801,-0.042237,0.608683,-0.176903
3,(Eggs),(Cheese),0.438095,0.501587,0.298413,0.681159,1.358008,0.078670,1.563203,0.469167
4,(Meat),(Cheese),0.476190,0.501587,0.323810,0.680000,1.355696,0.084958,1.557540,0.500891
5,(Cheese),(Meat),0.501587,0.476190,0.323810,0.645570,1.355696,0.084958,1.477891,0.526414
6,(Milk),(Cheese),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148,0.350053
7,(Cheese),(Milk),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148,0.350053
8,(Cheese),(NaN),0.501587,0.869841,0.393651,0.784810,0.902245,-0.042651,0.604855,-0.178565
9,(Wine),(Cheese),0.438095,0.501587,0.269841,0.615942,1.227986,0.050098,1.297754,0.330409


Provide explanation about __antecedent support__, __consequent support__, __support__, __confidence__, __lift__, __leverage__, __conviction__, __conviction__ and the interpretation from the case above (please use text section)

In the context of association rule mining for the given store dataset, several key metrics play a crucial role in helping to understand the relationships between items in transactions and how they might be related to customer behavior. These metrics offer valuable insights into the buying patterns of customers, allowing stores to make data-driven decisions.

Antecedent support refers to the proportion of transactions that contain the antecedent (the items on the left side of the rule). For example, if we examine the rule “{Bread} -> {Wine}”, the antecedent support would be the fraction of transactions in the dataset where Bread is purchased. This helps to measure how popular or common an item is in general.

Consequent support is the proportion of transactions that contain the consequent (the items on the right side of the rule). In the same rule “{Bread} -> {Wine}”, the consequent support would represent the fraction of transactions where Wine appears. This indicates how frequently the item on the right side of the rule is purchased by customers.

Support is the proportion of transactions where both the antecedent and consequent appear together. For the rule “{Bread} -> {Wine}”, support indicates how often both Bread and Wine are bought in the same transaction. A higher support indicates a stronger overall relationship between these items in customer purchases.

Confidence is a measure of the likelihood that the consequent will occur given that the antecedent has already occurred. It is calculated as the ratio of the support of both the antecedent and consequent to the support of the antecedent. For example, for the rule “{Bread} -> {Wine}”, confidence reflects the likelihood that a customer who buys Bread will also purchase Wine. Higher confidence suggests a strong likelihood that one item leads to the purchase of another.

Lift is a metric that shows the strength of the association between the antecedent and consequent compared to what would be expected by chance. It is calculated as the ratio of the support of the rule to the product of the supports of the antecedent and the consequent. If the lift value is greater than 1, it indicates that the antecedent and consequent appear together more frequently than would be expected by random chance, suggesting a meaningful relationship between the items.

Leverage measures the difference between the observed frequency of the antecedent and consequent appearing together and the expected frequency if the two items were independent of each other. Higher leverage values suggest a stronger relationship, as they indicate that the actual occurrence of both items together exceeds the random occurrence.

Conviction is a measure of the strength of the rule, indicating how likely the antecedent is to occur without the consequent. This metric compares the observed frequency of the antecedent and consequent with the frequency that would be expected if the items were independent. A higher conviction value suggests that the antecedent and consequent have a strong positive association, meaning the occurrence of one item makes the occurrence of the other much more likely.

Together, these metrics help businesses understand which products are often purchased together, the strength of these associations, and how customers’ buying behaviors can influence each other. By analyzing these relationships, stores can tailor their marketing strategies, optimize product placement, and improve cross-selling opportunities, ultimately leading to better customer experience and increased sales. Understanding the likelihood of certain items being bought together also helps in inventory management and promotional campaign planning.


Source:

Agrawal, R., & Srikant, R. (1994). Fast Algorithms for Mining Association Rules. Proceedings of the 20th International Conference on Very Large Data Bases (VLDB), 487-499.

Han, J., Kamber, M., & Pei, J. (2011). Data Mining: Concepts and Techniques (3rd Edition). Elsevier.

Marr, B. (2016). Big Data in Practice. Wiley.

https://chatgpt.com/c/67482770-9cd0-800b-b300-8d2b1805dc68
